该程序用来读入胶球算符，构建胶球两点关联函数，并通过变分法来得到优化算符。注意a1pp算符需要做真空减除。我们将考虑不同的减除方案。

In [63]:
from mpi4py import MPI
import timeit
import numpy as np
from scipy import signal
import heat as ht
import torch

In [40]:
path = "/CLQCD/configs/purgauge/aniso5beta30l24t144/"
files = [path + "glb_%04i.dat"%i for i in range(1,100)]
nx,nt=8,96
nconf = len(files)

In [55]:
choice_channel={"a1pp":(0,24),"a2pp":(24,48),"e_pp":(48,96),"t1pp":(96,168),"t2pp":(168,240),
"a1pm":(240,264),"a2pm":(264,288),"e_pm":(288,336),"t1pm":(336,408),"t2pm":(408,480),
"a1mp":(480,504),"a2mp":(504,516),"e_mp":(528,576),"t1mp":(576,648),"t2mp":(648,720),
"a1mm":(720,744),"a2mm":(744,768),"e_mm":(768,816),"t1mm":(816,888),"t2mm":(888,960)}

In [43]:
tmp_data = np.zeros((nconf,nt,998))

In [44]:
j=0
for i in files:
    tmp_data[j,...] =np.fromfile(i).reshape(nt,998) 

In [53]:
np.save("glue_op_nx8nt96xi5beta30.npy",tmp_data[:,:,:960])

In [78]:
glue_op = tmp_data[:,:,choice_channel["a1pp"][0]:choice_channel["a1pp"][1]]

In [79]:
vacuue = np.average(glue_op,0)

In [80]:
nop = vacuue.shape[1]

In [81]:
glue_op = glue_op - vacuue[np.newaxis,:,:]

In [82]:
glue_cor = np.zeros((nconf,nt,nop,nop))

In [83]:
for i in range(nt):
    glue_op_shift = np.roll(glue_op,shift=-i,axis=1)
    glue_cor[:,i,:,:] = np.einsum("ijk,ijm->ikm",glue_op,glue_op_shift)

In [84]:
glue_cor_ave = np.average(glue_cor,0)

In [85]:
glue_cor_ave[:,0,0]

array([103.00654573, 103.00335065, 103.00166798, 102.99986647,
       102.99838758, 102.99782086, 102.99689213, 102.99691241,
       102.99631625, 102.99587518, 102.99604706, 102.99590103,
       102.9965758 , 102.99552279, 102.9954211 , 102.99511355,
       102.99519889, 102.99644906, 102.99694651, 102.99786477,
       102.99804461, 102.99745689, 102.99722745, 102.99555767,
       102.99523687, 102.99404237, 102.99222867, 102.99190421,
       102.99192103, 102.99232512, 102.9920327 , 102.99214419,
       102.99190204, 102.99135025, 102.99183875, 102.99174878,
       102.9922303 , 102.99226504, 102.99170063, 102.99173608,
       102.99190345, 102.99218736, 102.99179201, 102.99138676,
       102.99076079, 102.9900633 , 102.99015462, 102.9900087 ,
       102.9900165 , 102.9900087 , 102.99015462, 102.9900633 ,
       102.99076079, 102.99138676, 102.99179201, 102.99218736,
       102.99190345, 102.99173608, 102.99170063, 102.99226504,
       102.9922303 , 102.99174878, 102.99183875, 102.99

In [66]:
a = np.arange(1,10)
b = np.arange(10,1,-1)
c = signal.convolve(a,b)

In [71]:
np.roll(a,shift=0)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])